<a href="https://colab.research.google.com/github/Sarik123-long/Brain-tumor-detection/blob/main/Brain__tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()  # Upload the zip file


Saving archive (15).zip to archive (15).zip


In [2]:
import zipfile
import os

# Get the actual uploaded file name from the 'uploaded' dictionary
# Assuming only one file was uploaded in the previous cell
zip_path = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("brain_tumor_dataset")

# Check extracted folders
os.listdir("brain_tumor_dataset")

['yes', 'brain_tumor_dataset', 'no']

In [15]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

data_dir = "brain_tumor_dataset" # Corrected directory name
categories = ['yes', 'no']

X = []
y = []

for category in categories:
    folder = os.path.join(data_dir, category) # Construct the full path to the category folder
    label = categories.index(category)

    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 128))
            X.append(img)
            y.append(label)

X = np.array(X).reshape(-1, 128, 128, 1) / 255.0
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dataset Loaded:", X_train.shape, y_train.shape)

Dataset Loaded: (202, 128, 128, 1) (202, 2)


In [16]:
from tensorflow.keras import layers, models

def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(128 * 16 * 16, activation="relu", input_dim=100))
    model.add(layers.Reshape((16, 16, 128)))
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(1, kernel_size=3, padding="same", activation="tanh"))
    return model


In [17]:
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=(128, 128, 1), padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model


In [18]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002), metrics=['accuracy'])

    discriminator.trainable = False
    gan_input = layers.Input(shape=(100,))
    img = generator(gan_input)
    valid = discriminator(img)
    gan = models.Model(gan_input, valid)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002))
    return gan


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,322 (12.60 MB)

 Trainable params: 3,304,322 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [29]:
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np # Import numpy
from tensorflow.keras.applications import MobileNetV2 # Import MobileNetV2 again
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense # Import layers again
from tensorflow.keras.models import Model # Import Model again


# Convert grayscale images to 3 channels by repeating the single channel 3 times
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

# Redefine and recompile the model before training
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Use the converted RGB images for training and validation
model.fit(X_train_rgb, y_train, validation_data=(X_test_rgb, y_test), epochs=100, callbacks=[early_stop])

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 464ms/step - accuracy: 0.5891 - loss: 0.7071 - val_accuracy: 0.8039 - val_loss: 0.4400
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 221ms/step - accuracy: 0.8849 - loss: 0.3155 - val_accuracy: 0.8824 - val_loss: 0.3662
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.8998 - loss: 0.2517 - val_accuracy: 0.8627 - val_loss: 0.3843
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - accuracy: 0.9751 - loss: 0.1153 - val_accuracy: 0.7843 - val_loss: 0.3908
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 267ms/step - accuracy: 0.9668 - loss: 0.1164 - val_accuracy: 0.8235 - val_loss: 0.4058
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.9895 - loss: 0.0664 - val_accuracy: 0.8039 - val_loss: 0.4252
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 275ms/step - accuracy: 0.9945 - loss: 0.0537 - val_accuracy: 0.8431 - val_loss: 0.4358


In [30]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2)


In [31]:
loss, acc = model.evaluate(X_test_rgb, y_test) # Use X_test_rgb for evaluation
print(f"Test Accuracy: {acc*100:.2f}%")

model.save("brain_tumor_cnn_model.h5")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.8903 - loss: 0.3304


Test Accuracy: 88.24%


In [34]:
!pip install gradio

import gradio as gr
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

# Load the trained model
model = load_model("brain_tumor_cnn_model.h5")

# Define the categories
categories = ['No Tumor', 'Tumor'] # Assuming 'yes' corresponds to Tumor and 'no' to No Tumor based on the data loading

def predict_tumor(image):
    """
    Predicts whether an image contains a brain tumor.

    Args:
        image: A PIL Image object.

    Returns:
        A dictionary with class labels as keys and prediction probabilities as values.
    """
    if image is None:
        return {category: 0.0 for category in categories} # Return 0 probability for all categories if no image is uploaded

    # Preprocess the image
    img = np.array(image.resize((128, 128))) # Resize to match model input size
    if img.shape[-1] == 4: # Remove alpha channel if present
      img = img[..., :3]
    if len(img.shape) == 2: # Convert grayscale to 3 channels if needed
      img = np.repeat(img[..., np.newaxis], 3, axis=-1)
    img = img / 255.0 # Normalize
    img = np.expand_dims(img, axis=0) # Add batch dimension

    # Make prediction
    predictions = model.predict(img)[0]

    # Return predictions as a dictionary
    return {categories[i]: float(predictions[i]) for i in range(len(categories))}

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_tumor,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Brain Tumor Classification",
    description="Upload a brain MRI image to classify if a tumor is present."
)

# Launch the interface
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://34bdcb09abcb9845b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
